In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import csv
import math
import matplotlib.pyplot as plt

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
num_dir = 13
x=pd.Series(np.linspace(0,180,num_dir))
y=pd.Series(np.linspace(np.nan,np.nan,num_dir))
Dir = pd.concat([x, y], axis=1)

print(Dir)


vessel_type = "Tank"   
bow_shape = "CONV"
cond = "Load"
trans_wind_area = 100  #m**3
long_wind_area = 500 ##m**3
curr_area = 0
LBP = 100
B = 20
t = 10 # make draft
WD = 0
rho_water= 1025 # kg/m**3
rho_air = 1.19 # kg/m**3


In [ ]:
### Defining Functions

def Inter_Dir(OCIMF_frame,direction_frame):
   a = []
   b = []
   WIFX = [WIFX , Dir]
   WIFX = pd.concat(WIFX)
   WIFX = WIFX.sort_values(by=0,ignore_index=True)
   WIFX = WIFX.interpolate(method = "linear")    
    
    
    
    
    
    
    
    
    
    OCIMF_frame = [frame1 , frame2]
    OCIMF_frame = pd.concat(frame1)
    OCIMF_frame = frame1.sort_values(by=0,ignore_index=True)
    OCIMF_frame = frame1.interpolate(method = "linear")
    for i in range(len(frame1.loc[0:])):
        for j in range(0, len(frame2[0])):
            if Dir.loc[j,0] == frame1.loc[i,0]:
                a.append(frame2.loc[j,0])
                b.append(frame1.loc[i,1])
    frame1 = pd.DataFrame({0: a, 1: b})            
    for i in range(len(frame1.loc[0:])):
        if math.isnan(frame1.loc[i,1]) == True:
            frame1=frame1.drop(frame1.index[[i]])
     
              
         
#### WIFX 
##Adding interpolated values to tables

a = []
b = []
WIFX = [WIFX , Dir]
WIFX = pd.concat(WIFX)
WIFX = WIFX.sort_values(by=0,ignore_index=True)
WIFX = WIFX.interpolate(method = "linear")


for i in range(len(WIFX.loc[0:])):
    for j in range(0, len(Dir[0])):
        if Dir.loc[j,0] == WIFX.loc[i,0]:
            a.append(Dir.loc[j,0])
            b.append(WIFX.loc[i,1])
WIFX = pd.DataFrame({0: a, 1: b})            

for i in range(len(WIFX.loc[0:])):
    if math.isnan(WIFX.loc[i,1]) == True:
        WIFX=WIFX.drop(WIFX.index[[i]])

WIFX = WIFX.drop_duplicates()    

WIFX[1] = WIFX[1]*((1/2)*rho_air*trans_wind_area)


In [ ]:
# Selects OCIMF List of Wind Coefficient based on bow shape and loading condition (Loaded or Ballasted)
# from raw OCIMF 2008 Curves data retrieved with an online plotter

if vessel_type == "Tank":
    if bow_shape == "CONV" and cond == "Load":    
        WIFX = pd.read_csv('/kaggle/input/ocimf-coefficients/Long_lWiConvLoad.csv',header=None)
        WIFY = pd.read_csv('/kaggle/input/ocimf-coefficients/LateralWindLoaded.csv',header=None)
        WIRZ = pd.read_csv('/kaggle/input/ocimf-coefficients/WindYawLoaded.csv',header=None)
    elif bow_shape == "CONV" and cond == "Bal":   
        WIFX = pd.read_csv('/kaggle/input/ocimf-coefficients/Long_lWindConvBowBal.csv',header=None)
        WIFY = pd.read_csv('/kaggle/input/ocimf-coefficients/LateralWindBall.csv',header=None)
        WIRZ = pd.read_csv('/kaggle/input/ocimf-coefficients/WindYawBal.csv',header=None)
    elif bow_shape == "CYL":    
        WIFX = pd.read_csv('/kaggle/input/ocimf-coefficients/Long_lWindCylBow.csv',header=None)
elif vessel_type == "LNGC" and shape == "Pris":
        WIFX = pd.read_csv('/kaggle/input/ocimf-coefficients/LNGC-Long_lWindPris(1).csv',header=None)
        WIFY = pd.read_csv('/kaggle/input/ocimf-coefficients/LNGC-LATWINDPRIS.csv',header=None)
        WIRZ = pd.read_csv('/kaggle/input/ocimf-coefficients/LNGC-WINDWAYPRISM.csv',header=None)
elif vessel_type == "LNGC" and shape == "Sphere":
        WIFX = pd.read_csv('/kaggle/input/ocimf-coefficients/LNGC-LONG_LWINDSPHER.csv',header=None)
        WIFY = pd.read_csv('/kaggle/input/ocimf-coefficients/LNGC-LATWINDSPHER.csv',header=None)
        WIRZ = pd.read_csv('/kaggle/input/ocimf-coefficients/LNGC-WINDYAWSPHER.csv',header=None)

# Replacing raw data for WIFY and WIRZ at 0 and 180 degrees to correct for raw data discrepencies at ends
# where values should equal 0 (eg. WIFY@180=0)

WIFY.loc[0,0]=180
WIFY.loc[0,1]=0
WIFY.loc[len(WIFY)-1,0]=0
WIFY.loc[len(WIFY)-1,1]=0
WIFX.loc[len(WIFX)-1,0]=0    #replacing last direction to equal 0 to correct for scanning error
WIFX.loc[0,0]=180

WIRZ.loc[0,0]=180
WIRZ.loc[0,1]=0
WIRZ.loc[len(WIRZ)-1,0]=0
WIRZ.loc[len(WIRZ)-1,1]=0

#### WIFX 
##Adding interpolated values to tables
a = []
b = []
WIFX2 = pd.concat([WIFX, Dir])
WIFX2 = WIFX2.sort_values(by=0,ignore_index=True)
WIFX2 = WIFX2.set_index(0) # Added PM 02 Jan 2021
WIFX2 = WIFX2.interpolate(method = "polynomial", order=1) # Modified by PM 01 Jab 2021
WIFX2 = WIFX2.reset_index() # Added PM 02 Jan 2021
for i in range(len(WIFX2.loc[0:])):
    for j in range(0, len(Dir[0])):
        if Dir.loc[j,0] == WIFX2.loc[i,0]:
            a.append(Dir.loc[j,0])
            b.append(WIFX2.loc[i,1])
WIFX2 = []
WIFX2 = pd.DataFrame({0: a, 1: b})            
for i in range(len(WIFX2.loc[0:])):
    if math.isnan(WIFX2.loc[i,1]) == True:
        WIFX2=WIFX2.drop(WIFX2.index[[i]])
        WIFX2 = WIFX2.drop_duplicates()
# Plot and compare WIFX raw and interpolated Coefficients
plt.plot(WIFX[0],WIFX[1], label = 'Raw')
plt.plot(WIFX2[0],WIFX2[1], label = 'Interpolated & Purged')
plt.title('OCIMF Surge Wind Coefficients')
plt.xlabel('Direction, degrees')
plt.ylabel('Coefficient')
plt.legend()
plt.savefig('WIFX-2.png')
plt.show()
WIFX2[1] = WIFX2[1]*((1/2)*rho_air*trans_wind_area)
WIFX = []
WIFX = WIFX2

#### WIFY 
##Adding interpolated values to tables
a = []
b = []
WIFY2 = pd.concat([WIFY , Dir])
WIFY2 = WIFY2.sort_values(by=0,ignore_index=True)
WIFY2 = WIFY2.set_index(0) # Added PM 02 Jan 2021
WIFY2 = WIFY2.interpolate(method = "polynomial", order=1)
WIFY2 = WIFY2.reset_index() # Added PM 02 Jan 2021
for i in range(len(WIFY2.loc[0:])):
    for j in range(0, len(Dir[0])):
        if Dir.loc[j,0] == WIFY2.loc[i,0]:
            a.append(Dir.loc[j,0])
            b.append(WIFY2.loc[i,1])
WIFY2 = []
WIFY2 = pd.DataFrame({0: a, 1: b})
for i in range(len(WIFY2.loc[0:])):
    if math.isnan(WIFY2.loc[i,1]) == True:
        WIFY2=WIFY2.drop(WIFY2.index[[i]])
WIFY2 = WIFY2.drop_duplicates()
# Plot and compare WIFY raw and interpolated Coefficients
plt.plot(WIFY[0],WIFY[1], label = 'Raw')
plt.plot(WIFY2[0],WIFY2[1], label = 'Interpolated & Purged')
plt.title('OCIMF Sway Wind Coefficients')
plt.xlabel('Direction, degrees')
plt.ylabel('Coefficient')
plt.legend()
plt.savefig('WIFY-1.png')
plt.show()
WIFY2[1] = WIFY2[1]*((1/2)*rho_air*long_wind_area)
WIFY = []
WIFY = WIFY2

#### WIRZ 
##Adding interpolated values to tables
a = []
b = []
WIRZ2 = pd.concat([WIRZ , Dir])
WIRZ2 = WIRZ2.sort_values(by=0,ignore_index=True)
WIRZ2 = WIRZ2.set_index(0) # Added PM 02 Jan 2021
WIRZ2 = WIRZ2.interpolate(method = "polynomial", order=1)
WIRZ2 = WIRZ2.reset_index() # Added PM 02 Jan 2021
for i in range(len(WIRZ2.loc[0:])):
    for j in range(0, len(Dir[0])):
        if Dir.loc[j,0] == WIRZ2.loc[i,0]:
            a.append(Dir.loc[j,0])
            b.append(WIRZ2.loc[i,1])
WIRX2 = []
WIRZ2 = pd.DataFrame({0: a, 1: b}) 
for i in range(len(WIRZ2.loc[0:])):
    if math.isnan(WIRZ2.loc[i,1]) == True:
        WIRZ2=WIRZ2.drop(WIRZ2.index[[i]])
WIRZ2 = WIRZ2.drop_duplicates()
# Plot and compare WIFY raw and interpolated Coefficients
plt.plot(WIRZ[0],WIRZ[1], label = 'Raw')
plt.plot(WIRZ2[0],WIRZ2[1], label = 'Interpolated & Purged')
plt.title('OCIMF Yaw Wind Coefficients')
plt.xlabel('Direction, degrees')
plt.ylabel('Coefficient')
plt.legend()
plt.savefig('WIRZ-1.png')
plt.show()
WIRZ2[1] = WIRZ2[1]*((1/2)*rho_air*long_wind_area*LBP)
WIRZ = []
WIRZ = WIRZ2

print('Surge Force Factors')
print(WIFX)
print('Sway Force Factors')
print(WIFY)
print('Yam Moment Factors')
print(WIRZ)




   
